# Scrape leslibraires.ca for book data

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
# function to extract data from a book's page on the website
def get_book_info(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    data = {'title': soup.find('h1', {'class': 'book-title'}).text,
            'author': soup.find('a', {'class': 'book-meta-author-name'}).text.replace('\n', ''),
            'ISBN':  soup.find('div', {'id': 'sel-buy-box'}).find('span', {'class': 'buy-box--isbn'}).text[6:],
            'summary': soup.find('div', {'class': 'book-summary'}).text.replace('\n', ''),
            'image': soup.find('img', {'class': 'book-image'})['src']}
    return data

In [7]:
# get data about most popular books on the website
full_data = []
for page in range(1000):
    print(page)
    page = requests.get('https://www.leslibraires.ca/categorie/litterature-LIT/?tri=plus-populaires&ic=25&p={}'.format(page))
    soup = BeautifulSoup(page.content, 'html.parser')
    for book in soup.find_all('a', {'class': 'book-title'}):
        base_url = 'https://www.leslibraires.ca'
        url = base_url + book['href']
        try:
            full_data.append(get_book_info(url))
        except Exception as e:
            print(e)

0
1
2
'NoneType' object has no attribute 'find'
3
'NoneType' object has no attribute 'find'
4
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
5
6
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
7
'NoneType' object has no attribute 'find'
8
'NoneType' object has no attribute 'find'
9
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
10
'NoneType' object has no attribute 'find'
11
'NoneType' object has no attribute 'find'
12
'NoneType' object has no attribute 'text'
13
'NoneType' object has no attribute 'find'
14
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
15
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
16
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 

103
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
104
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
105
'NoneType' object has no attribute 'find'
106
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
107
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
108
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
109
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
110
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
111
'NoneType' object has no attribute 'find'
112
'NoneType' object has no attribute 'find'
113
'NoneType' object has no attribute 'find'
'NoneType' object has no attribu

'NoneType' object has no attribute 'find'
185
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
186
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
187
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
188
'NoneType' object has no attribute 'find'
189
'NoneType' object has no attribute 'find'
190
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
191
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
192
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
19

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
249
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
250
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
251
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
252
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object 

309
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
310
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
311
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
312
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
313
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
314
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
315
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
316
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'N

...

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
831
'NoneType' object has no attribute 'find'
832
'NoneType' object has no attribute 'find'
833
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
834
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
835
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
836
837
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
838
'NoneType' object has no attribute 'find'
'N

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
869
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
870
871
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
872
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
873
'NoneType' object has no attribute 'find'
874
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
875
'NoneT

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
929
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
930
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
931
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
932
'NoneType' object has no attribute 'find'
'NoneType' object 

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
959
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
960
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
961
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
962
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object 

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
993
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
994
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
995
'NoneType' object has no attribute 'find'
'NoneType' object has 

In [10]:
len(full_data)

16523

In [9]:
import pickle

with open('books_long.pkl', 'wb') as f:
    pickle.dump(full_data, f)

In [12]:
import pickle

with open('books.pkl', 'rb') as f:
    book_data = pickle.load(f)

In [13]:
book_data

[{'title': 'Kukum',
  'author': 'Michel Jean',
  'ISBN': '9782764813447',
  'summary': "Ce roman retrace le parcours d'Almanda Siméon, une orpheline qui va partager sa vie avec les Innus de Pekuakami. Amoureuse d'un jeune Innu, elle réussira à se faire accepter. Elle apprendra l'existence nomade et la langue, et brisera les barrières imposées aux femmes autochtones. Almanda et sa famille seront confrontées à la perte de leurs terres et subiront l'enfermement des réserves et la violence des pensionnats. Racontée sur un ton intimiste, l'histoire de cette femme, qui se déroule sur un siècle, exprime l'attachement aux valeurs ancestrales des Innus et au besoin de liberté qu'éprouvent les peuples nomades, encore aujourd'hui."},
 {'title': "El Kapoutchi T.3 : L'infâme piège de Noël",
  'author': 'Pascale Richard',
  'ISBN': '9782897144043',
  'summary': "Par un matin un peu frisquet de décembre, El Kapoutchi erre chez lui en pyjama, à la recherche d'un plan machiavélique. Il lui faut frapper

# Encode description of books using tensorflow

In [ ]:
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
for element in book_data:
    element['vector'] = embed(element['summary'])[0]